In [0]:
!pip install -U tensorflow
!pip install nltk
!pip install keras

     |████████████████████████████████| 421.8MB 36kB/s 
     |████████████████████████████████| 450kB 51.8MB/s 
     |████████████████████████████████| 3.8MB 46.1MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstall

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.1.0)


In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

import string

from nltk.corpus import stopwords
from nltk.probability import FreqDist
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [0]:

positive=pd.read_table("polarity.pos.csv", encoding="UTF8", header=None, names=['Review'])
positive['Sentiment'] = 1
positive


,Review,Sentiment
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
5326,both exuberantly romantic and serenely melanch...,1
5327,mazel tov to a film about a family's joyous li...,1
5328,standing in the shadows of motown is the best ...,1
5329,it's nice to see piscopo again after all these...,1


In [0]:
positive.shape

(5331, 2)

In [0]:
negative=pd.read_table("polarity.neg.csv", encoding="UTF8", header=None, names=['Review'])
negative["Sentiment"] = 0
negative

,Review,Sentiment
0,"simplistic , silly and tedious.",0
1,"it's so laddish and juvenile , only teenage bo...",0
2,exploitative and largely devoid of the depth o...,0
3,[garbus] discards the potential for pathologic...,0
4,a visually flashy but narratively opaque and e...,0
...,...,...
5326,a terrible movie that some people will neverth...,0
5327,there are many definitions of 'time waster' bu...,0
5328,"as it stands , crocodile hunter has the hurrie...",0
5329,the thing looks like a made-for-home-video qui...,0


In [0]:
negative.shape

(5331, 2)

## OCZYSZCZANIE DANYCH

In [0]:
def clean_col(col):
    col = col.strip()
    col = col.replace(","," ")
    col = col.replace("''"," ")
    col = col.replace("'s"," ")
    col = col.replace("."," ")
    col = col.replace("-"," ")
    col = col.replace("_"," ")
    col = col.replace("["," ")
    col = col.replace("]"," ")
    col = col.replace("--"," ")
    col = col.replace("`"," ")
    col = col.replace(" '", " ")    

    col = col.lower()
    return col

In [0]:
positive['Review'] = [clean_col(c) for c in positive['Review']]
negative['Review'] = [clean_col(c) for c in negative['Review']]
print(positive)
print(negative)

                                                 Review  Sentiment
0     the rock is destined to be the 21st century  n...          1
1     the gorgeously elaborate continuation of " the...          1
2                        effective but too tepid biopic          1
3     if you sometimes like to go to the movies to h...          1
4     emerges as something rare   an issue movie tha...          1
...                                                 ...        ...
5326  both exuberantly romantic and serenely melanch...          1
5327  mazel tov to a film about a family  joyous lif...          1
5328  standing in the shadows of motown is the best ...          1
5329  it  nice to see piscopo again after all these ...          1
5330  provides a porthole into that noble   tremblin...          1

[5331 rows x 2 columns]
                                                 Review  Sentiment
0                       simplistic   silly and tedious           0
1     it  so laddish and juvenile   o

In [0]:
dane=pd.concat([positive,negative],ignore_index=True)
type(dane)

pandas.core.frame.DataFrame

In [0]:
def remove_punctuation(text):
  no_punct="".join([c for c in text if c not in string.punctuation])
  return no_punct

In [0]:
dane['Review']=dane['Review'].apply(lambda x: remove_punctuation(x))
dane['Review'].head()

0    the rock is destined to be the 21st century  n...
1    the gorgeously elaborate continuation of  the ...
2                       effective but too tepid biopic
3    if you sometimes like to go to the movies to h...
4    emerges as something rare   an issue movie tha...
Name: Review, dtype: object

In [0]:
tokenizer= RegexpTokenizer(r'\w+')

In [0]:
dane['Review']=dane['Review'].apply(lambda x: tokenizer.tokenize(x.lower()))
dane['Review'].head()

0    [the, rock, is, destined, to, be, the, 21st, c...
1    [the, gorgeously, elaborate, continuation, of,...
2                 [effective, but, too, tepid, biopic]
3    [if, you, sometimes, like, to, go, to, the, mo...
4    [emerges, as, something, rare, an, issue, movi...
Name: Review, dtype: object

In [0]:
def remove_stopwords(text):
  words=[w for w in text if w not in stopwords.words('english')]
  return words

In [0]:
dane['Review']=dane['Review'].apply(lambda x: remove_stopwords(x))
dane['Review'].head()

0    [rock, destined, 21st, century, new, conan, go...
1    [gorgeously, elaborate, continuation, lord, ri...
2                           [effective, tepid, biopic]
3    [sometimes, like, go, movies, fun, wasabi, goo...
4    [emerges, something, rare, issue, movie, hones...
Name: Review, dtype: object

In [0]:
lemmatizer=WordNetLemmatizer()

def word_lemmatizer(text):
  lem_text=[lemmatizer.lemmatize(i) for i in text]
  return lem_text

In [0]:
dane['Review'].apply(lambda x : word_lemmatizer(x))

0        [rock, destined, 21st, century, new, conan, go...
1        [gorgeously, elaborate, continuation, lord, ri...
2                               [effective, tepid, biopic]
3        [sometimes, like, go, movie, fun, wasabi, good...
4        [emerges, something, rare, issue, movie, hones...
                               ...                        
10657    [terrible, movie, people, nevertheless, find, ...
10658    [many, definition, time, waster, movie, must, ...
10659    [stand, crocodile, hunter, hurried, badly, cob...
10660      [thing, look, like, made, home, video, quickie]
10661                    [enigma, well, made, dry, placid]
Name: Review, Length: 10662, dtype: object

In [0]:
stemmer=PorterStemmer()

In [0]:
def word_stemmer(text):
  stem_text=" ".join([stemmer.stem(i) for i in text])
  return stem_text

In [0]:
dane['Review']=dane['Review'].apply(lambda x: word_stemmer(x))


# MIESZANIE DANYCH

In [0]:
dane=dane.sample(frac=1).reset_index(drop=True)
dane

,Review,Sentiment
0,outer space buff might love film other find pl...,0
1,serri want blend polit drama admir ambit bad h...,0
2,kid often appear read line incap convey emot,0
3,real snooz,0
4,like trap insid huge video game excit inan ima...,0
...,...,...
10657,stori could expand work compel singl featur cu...,0
10658,limp along squirm induc fish water formula goe...,0
10659,though excess tiresom uncertainti principl ver...,0
10660,chicago huge imagin success cast great credit ...,1


In [0]:
X, y = (dane['Review'].values, dane['Sentiment'].values)

In [0]:
print(X,y)

['outer space buff might love film other find pleasur intermitt'
 'serri want blend polit drama admir ambit bad help hand use stir ingredi also heavi one'
 'kid often appear read line incap convey emot' ...
 'though excess tiresom uncertainti principl verbal pretenti titl may hand redeem featur long discount abil bore'
 'chicago huge imagin success cast great credit well one terrif score attitud spare'
 'humor aspect jason x far entertain expect everyth els film tank'] [0 0 0 ... 0 1 0]


In [0]:
tk = Tokenizer(lower = True)
tk.fit_on_texts(X)
X_seq = tk.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=100, padding='post')


In [0]:
print(dane['Review'][3],X_seq[3])

real snooz [90, 5733]


In [0]:
len(dane)

10662

In [0]:
split_frac = 0.8
train_x = X_pad[0:int(split_frac*len(dane))]
train_y = y[0:int(split_frac*len(dane))]

remaining_x = X_pad[int(split_frac*len(dane)):]
remaining_y = y[int(split_frac*len(dane)):]

valid_x = remaining_x[0:int(len(remaining_x)*0.5)]
valid_y = remaining_y[0:int(len(remaining_y)*0.5)]

test_x = remaining_x[int(len(remaining_x)*0.5):]
test_y = remaining_y[int(len(remaining_y)*0.5):]

In [0]:
print(len(train_x))
print(len(train_y))
print(len(valid_x))
print(len(valid_y))
print(len(test_x))
print(len(test_y))

8529
8529
1066
1066
1067
1067


In [0]:
import torch
from torch.utils.data import DataLoader, TensorDataset
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
# dataloaders
batch_size = 50
# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [0]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()
print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 100])
Sample input: 
 tensor([[4100,  247,   81,  ...,    0,    0,    0],
        [4940,  547, 3612,  ...,    0,    0,    0],
        [ 194,   64, 9604,  ...,    0,    0,    0],
        ...,
        [ 218,    9,  471,  ...,    0,    0,    0],
        [ 908,   39,  325,  ...,    0,    0,    0],
        [ 358,  309,    0,  ...,    0,    0,    0]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
        1, 1])


In [0]:
from string import punctuation
import collections
print(punctuation)

all_text = ''.join([c for c in dane if c not in punctuation])

reviews_split = all_text.split('\n')
print ('Number of reviews :', len(reviews_split))

all_text2 = ' '.join(reviews_split)
# create a list of words
words = all_text2.split()

count_words=collections.Counter(words)

total_words = len(words)
sorted_words = count_words.most_common(total_words)

vocab_to_int = {w:i for i, (w,c) in enumerate(sorted_words)}

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Number of reviews : 1


# ***Modeeel***

In [0]:
# Instantiate the model w/ hyperparams
from __future__ import print_function


from keras.models import Sequential
from tensorflow.keras import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 100
hidden_dim = 256
n_layers = 2
net = SentimentLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)


from tensorflow.keras import layers

model = tf.keras.Sequential()

model.add(layers.Embedding(input_dim=15000 , output_dim=128))

model.add(layers.LSTM(128, dropout=0.2,recurrent_dropout=0.2))



model.add(layers.Dense(2, activation='sigmoid'))

model.summary()

SentimentLSTM(
  (embedding): Embedding(2, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)
Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 128)         1920000   
_________________________________________________________________
lstm_16 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 129       
Total params: 2,051,713
Trainable params: 2,051,713
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [0]:
model.fit(train_x, train_y,
          batch_size=32,
          epochs=3,
          validation_data=(valid_x, valid_y))
score, acc = model.evaluate(test_x, test_y,
                            batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

Train on 8529 samples, validate on 1066 samples
Epoch 1/3
8529/8529 [==============================] - 87s 10ms/sample - loss: 0.6939 - accuracy: 0.4949 - val_loss: 0.6930 - val_accuracy: 0.5075
Epoch 2/3
8529/8529 [==============================] - 85s 10ms/sample - loss: 0.6935 - accuracy: 0.4968 - val_loss: 0.6931 - val_accuracy: 0.5075
Epoch 3/3
1067/1067 [==============================] - 1s 1ms/sample - loss: 0.6929 - accuracy: 0.5145
Test score: 0.6929233240656687
Test accuracy: 0.5145267
